In [1]:
# import
import numpy as np
import gym
from gym import spaces
import random

import collections
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
class Grid(gym.Env):
    metadata = {'render.modes': ['console']}
    # action id
    XM = 0 # x minus
    XP = 1 # x plus
    YM = 2 # y minus
    YP = 3 # y plus
    
    def __init__(self, x_size=5):
        super(Grid, self).__init__()
        
        # size of 2D grid
        self.x_size = x_size

        # initialize the mapping status
        self.init_grid()
        
        # initialize the position of the agent
        self.init_agent()
        
        # define action space
        n_actions = 4 # LEFT, RIGHT, TOP, BOTTOM
        self.action_space = spaces.Discrete(n_actions)
        
        # define observation space (x and y coordinates)
        self.obs_low = np.zeros(2)
        self.obs_high = np.ones(2) * (self.x_size - 1)
        self.observation_space = spaces.Box(self.obs_low, self.obs_high)
    
    def init_agent(self, initial_pos=None):
        if initial_pos is not None:
            self.agent_pos = initial_pos
        else:
            self.agent_pos = [0, 0]
            self.agent_pos[0] = random.randrange(0, self.x_size)
            self.agent_pos[1] = random.randrange(0, self.x_size)

        self.grid_status[self.agent_pos[0], self.agent_pos[1]] = 1
        self.n_poi = self.x_size ** 2 - np.count_nonzero(self.grid_status)
    
    def init_grid(self):
        self.grid_status = np.zeros([self.x_size, self.x_size])
    
    def get_coverage(self):
        mapped_poi = (self.grid_status == 1).sum() - 1 # exclude initial pos
        return mapped_poi / self.n_poi

    def get_agent_obs(self):
        pos_x  = copy.deepcopy(self.agent_pos[0])
        pos_y  = copy.deepcopy(self.agent_pos[1])

        return [pos_x, pos_y]

    def reset(self, initial_pos=None):
        self.init_grid()
        self.init_agent(initial_pos)

        return self.get_agent_obs()
        
    def step(self, action): # i: index of the drone
        # original position
        org_x  = copy.deepcopy(self.agent_pos[0])
        org_y  = copy.deepcopy(self.agent_pos[1])

        # move the agent
        if action == self.XM:
            self.agent_pos[0] -= 1
        elif action == self.XP:
            self.agent_pos[0] += 1
        elif action == self.YM:
            self.agent_pos[1] -= 1
        elif action == self.YP:
            self.agent_pos[1] += 1
        else:
            raise ValueError("Received invalid action={} which is not part of the action space".format(action))
        
        # account for the boundaries of the grid (-2: out of the grid)
        if self.agent_pos[0] > self.x_size - 1 or self.agent_pos[0] < 0 or self.agent_pos[1] > self.x_size - 1 or self.agent_pos[1] < 0:
            self.agent_pos[0] = org_x
            self.agent_pos[1] = org_y

        # reward
        prev_status = self.grid_status[self.agent_pos[0], self.agent_pos[1]]
        if prev_status == 0:
            reward = 10
            self.grid_status[self.agent_pos[0], self.agent_pos[1]] = 1
        else:
            reward = 0
        
        # done
        mapped_poi = (self.grid_status == 1).sum() - 1 # exclude initial pos
        if mapped_poi == self.n_poi:
            done = True
        else:
            done = False
        
        return self.get_agent_obs(), reward, done

    def close(self):
        pass

In [16]:
class QTables():
    def __init__(self, observation_space, action_space, eps_start=1, eps_end=0.1, gamma=0.9, r=0.99, lr=0.1):
        self.observation_space = observation_space
        self.observation_length = observation_space.shape[0]
        self.size = int(self.observation_space.high[0] - self.observation_space.low[0]) + 1

        self.action_space = action_space
        self.action_values = [0, 1, 2, 3] # corresponding to the column numbers in q table
        self.action_num = len(self.action_values) # 4

        self.eps = eps_start  # current epsilon
        self.eps_end = eps_end # epsilon lower bound
        self.r = r  # decrement rate of epsilon
        self.gamma = gamma  # discount rate
        self.lr = lr  # learning rate

        self.q_table = np.zeros([self.size**2, self.action_num])
        self.q_table_count = np.zeros([self.size**2, self.action_num])

    # support function: convert the fov to the unique row number in the q table
    def obs_to_row(self, obs_array):
        return obs_array[0] * self.size + obs_array[1]
    
    def get_action(self, obs):
        if np.random.rand() < self.eps:
            action = random.choice(self.action_values)
            greedy = False
        else:
            obs_row = self.obs_to_row(obs)
            action = np.argmax(self.q_table[obs_row])
            greedy = True
        
        return action, greedy
    
    def update_eps(self):
        # update the epsilon
        if self.eps > self.eps_end: # lower bound
            self.eps *= self.r

    def train(self, obs, obs_next, action, reward, done):
        obs_row = self.obs_to_row(obs)
        obs_next_row = self.obs_to_row(obs_next)

        q_current = self.q_table[obs_row][action] # current q value
        q_next_max = np.max(self.q_table[obs_next_row]) # the maximum q value in the next state

        # update the q value
        if done:
            self.q_table[obs_row][action] = q_current + self.lr * reward
        else:
            self.q_table[obs_row][action] = q_current + self.lr * (reward + self.gamma * q_next_max - q_current)
        
        # update the count
        self.q_table_count[obs_row][action] += 1

### Fixed Initial Position

#### 10 x 10

In [ ]:
# records for each episode
time_steps = [] # number of time steps in total
epsilons = [] # epsilon at the end of each episode
greedy = [] # the ratio of greedy choices
trajectory = []
coverage = []

q_class = []

# parameters for training
train_episodes = 100000
size = 10
max_steps = size * 10

# initialize the environment and the q tables
env = Grid(x_size=size)
q = QTables(observation_space=env.observation_space, action_space=env.action_space, eps_start=1, eps_end=0, gamma=0.5, r=0.9999, lr=0.01)

# training
for episode in range(train_episodes):
    env.reset([0, 0])
    state = env.get_agent_obs()
    eps_tmp = q.eps

    greedy_count = 0
    epi_trajectory = []
    epi_step = 0
    epi_trajectory.append(env.get_agent_obs())

    for step in range(max_steps):
        action, greedy_tf = q.get_action(obs=state)
        next_state, reward, done = env.step(action)
        q.train(state, next_state, action, reward, done)

        greedy_count += greedy_tf * 1
        epi_trajectory.append(env.get_agent_obs())

        if done:
            epi_step = step
            break
    
        # update the observation
        state = next_state

    # record
    time_steps.append(epi_step + 1)
    epsilons.append(eps_tmp)
    greedy.append(greedy_count / (step + 1))
    q_class.append(copy.deepcopy(q))
    trajectory.append(epi_trajectory)
    coverage.append(env.get_coverage())

    # update epsilon
    q.update_eps()

    print(episode, time_steps[episode], epsilons[episode], greedy[episode], coverage[episode])

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Streaming output truncated to the last 5000 lines.
95000 1 7.481628134210545e-05 1.0 0.7171717171717171
95001 1 7.480879971397124e-05 1.0 0.7171717171717171
95002 1 7.480131883399984e-05 1.0 0.7171717171717171
95003 1 7.479383870211644e-05 1.0 0.7171717171717171
95004 1 7.478635931824623e-05 1.0 0.7171717171717171
95005 1 7.477888068231441e-05 1.0 0.7171717171717171
95006 1 7.477140279424618e-05 1.0 0.7171717171717171
95007 1 7.476392565396675e-05 1.0 0.7171717171717171
95008 1 7.475644926140136e-05 1.0 0.7171717171717171
95009 1 7.474897361647521e-05 1.0 0.7171717171717171
95010 1 7.474149871911357e-05 1.0 0.7171717171717171
95011 1 7.473402456924165e-05 1.0 0.7171717171717171
95012 1 7.472655116678473e-05 1.0 0.7171717171717171
95013 1 7.471907851166805e-05 1.0 0.7171717171717171
95014 1 7.471160660381688e-05 1.0 0.7171717171717171
95015 1 7.470413544315651e-05 1.0 0.7171717171717171
95016 1 7.46966650296122e-05 1.0 0.7171717171717171
95017 1 7.468919536310924e-05 1.0 0.7171717171717

In [ ]:
df = pd.DataFrame(q_class[100000-1].q_table, index=idx)
df.to_csv('qtable.csv')

In [ ]:
for i in [10000-1, 20000-1, 30000-1, 40000-1, 50000-1, 60000-1, 70000-1, 80000-1, 90000-1, 100000-1]:
    print(epsilons[i])

0.36789783621659083
0.13533528301106
0.049784578827750475
0.018313807263800937
0.00673693630423356
0.002478256438627614
0.0009116540068427968
0.0003353619969420021
0.0001233666151289349
4.538177213622325e-05


#### 10 x 10

In [ ]:
# records for each episode
time_steps = [] # number of time steps in total
epsilons = [] # epsilon at the end of each episode
greedy = [] # the ratio of greedy choices
trajectory = []
coverage = []

q_class = []

# parameters for training
train_episodes = 100000
size = 10
max_steps = size * 20

# initialize the environment and the q tables
env = Grid(x_size=size)
q = QTables(observation_space=env.observation_space, action_space=env.action_space, eps_start=1, eps_end=0, gamma=0.5, r=0.9999, lr=0.01)

# training
for episode in range(train_episodes):
    env.reset([0, 0])
    state = env.get_agent_obs()
    eps_tmp = q.eps

    greedy_count = 0
    epi_trajectory = []
    epi_step = 0
    epi_trajectory.append(env.get_agent_obs())

    for step in range(max_steps):
        action, greedy_tf = q.get_action(obs=state)
        next_state, reward, done = env.step(action)
        q.train(state, next_state, action, reward, done)

        greedy_count += greedy_tf * 1
        epi_trajectory.append(env.get_agent_obs())

        if done:
            epi_step = step
            break
    
        # update the observation
        state = next_state

    # record
    time_steps.append(epi_step + 1)
    epsilons.append(eps_tmp)
    greedy.append(greedy_count / (step + 1))
    q_class.append(copy.deepcopy(q))
    trajectory.append(epi_trajectory)
    coverage.append(env.get_coverage())

    # update epsilon
    q.update_eps()

    print(episode, time_steps[episode], epsilons[episode], greedy[episode], coverage[episode])

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Streaming output truncated to the last 5000 lines.
95000 1 7.481628134210545e-05 1.0 0.7070707070707071
95001 1 7.480879971397124e-05 1.0 0.7070707070707071
95002 1 7.480131883399984e-05 1.0 0.7070707070707071
95003 1 7.479383870211644e-05 1.0 0.7070707070707071
95004 1 7.478635931824623e-05 1.0 0.7070707070707071
95005 1 7.477888068231441e-05 1.0 0.7070707070707071
95006 1 7.477140279424618e-05 1.0 0.7070707070707071
95007 1 7.476392565396675e-05 1.0 0.7070707070707071
95008 1 7.475644926140136e-05 1.0 0.7070707070707071
95009 1 7.474897361647521e-05 1.0 0.7070707070707071
95010 1 7.474149871911357e-05 1.0 0.7070707070707071
95011 1 7.473402456924165e-05 1.0 0.7070707070707071
95012 1 7.472655116678473e-05 1.0 0.7070707070707071
95013 1 7.471907851166805e-05 1.0 0.7070707070707071
95014 1 7.471160660381688e-05 1.0 0.7070707070707071
95015 1 7.470413544315651e-05 1.0 0.7070707070707071
95016 1 7.46966650296122e-05 1.0 0.7070707070707071
95017 1 7.468919536310924e-05 1.0 0.7070707070707

In [ ]:
df = pd.DataFrame(q_class[80000-1].q_table, index=idx)
df.to_csv('qtable.csv')

Smaller Learning Rate

In [ ]:
# records for each episode
time_steps = [] # number of time steps in total
epsilons = [] # epsilon at the end of each episode
greedy = [] # the ratio of greedy choices
trajectory = []
coverage = []

q_class = []

# parameters for training
train_episodes = 100000
size = 10
max_steps = size * 20

# initialize the environment and the q tables
env = Grid(x_size=size)
q = QTables(observation_space=env.observation_space, action_space=env.action_space, eps_start=1, eps_end=0, gamma=0.5, r=0.9999, lr=0.001)

# training
for episode in range(train_episodes):
    env.reset([0, 0])
    state = env.get_agent_obs()
    eps_tmp = q.eps

    greedy_count = 0
    epi_trajectory = []
    epi_trajectory.append(env.get_agent_obs())

    for step in range(max_steps):
        action, greedy_tf = q.get_action(obs=state)
        next_state, reward, done = env.step(action)
        q.train(state, next_state, action, reward, done)

        greedy_count += greedy_tf * 1
        epi_trajectory.append(env.get_agent_obs())

        if done:
            break
    
        # update the observation
        state = next_state

    # record
    time_steps.append(len(epi_trajectory)-1)
    epsilons.append(eps_tmp)
    greedy.append(greedy_count / (step + 1))
    q_class.append(copy.deepcopy(q))
    trajectory.append(epi_trajectory)
    coverage.append(env.get_coverage())

    # update epsilon
    q.update_eps()

    print(episode, time_steps[episode], epsilons[episode], greedy[episode], coverage[episode])

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Streaming output truncated to the last 5000 lines.
40565 200 0.017305985779238833 0.98 0.25252525252525254
40566 200 0.017304255180660907 0.995 0.24242424242424243
40567 200 0.01730252475514284 0.96 0.36363636363636365
40568 200 0.017300794502667326 0.985 0.25252525252525254
40569 200 0.017299064423217058 0.97 0.3333333333333333
40570 200 0.017297334516774735 0.985 0.26262626262626265
40571 200 0.01729560478332306 0.98 0.2828282828282828
40572 200 0.017293875222844726 0.975 0.32323232323232326
40573 200 0.01729214583532244 0.98 0.32323232323232326
40574 200 0.01729041662073891 0.99 0.2828282828282828
40575 200 0.01728868757907684 0.965 0.30303030303030304
40576 200 0.01728695871031893 0.985 0.25252525252525254
40577 200 0.0172852300144479 0.97 0.35353535353535354
40578 200 0.017283501491446456 0.985 0.2828282828282828
40579 200 0.01728177314129731 0.99 0.32323232323232326
40580 200 0.017280044963983183 0.97 0.31313131313131315
40581 200 0.017278316959486784 0.99 0.2828282828282828
4058

KeyboardInterrupt: ignored

More episodes

In [18]:
# records for each episode
time_steps = [] # number of time steps in total
epsilons = [] # epsilon at the end of each episode
greedy = [] # the ratio of greedy choices
trajectory = []
coverage = []

q_class = []

# parameters for training
train_episodes = 200000
size = 10
max_steps = size * 20

# initialize the environment and the q tables
env = Grid(x_size=size)
q = QTables(observation_space=env.observation_space, action_space=env.action_space, eps_start=1, eps_end=0, gamma=0.5, r=0.9999, lr=0.01)

# training
for episode in range(train_episodes):
    env.reset([0, 0])
    state = env.get_agent_obs()
    eps_tmp = q.eps

    greedy_count = 0
    epi_trajectory = []
    epi_step = 0
    epi_trajectory.append(env.get_agent_obs())

    for step in range(max_steps):
        action, greedy_tf = q.get_action(obs=state)
        next_state, reward, done = env.step(action)
        q.train(state, next_state, action, reward, done)

        greedy_count += greedy_tf * 1
        epi_trajectory.append(env.get_agent_obs())

        if done:
            epi_step = step
            break
    
        # update the observation
        state = next_state

    # record
    time_steps.append(len(epi_trajectory)-1)
    epsilons.append(eps_tmp)
    greedy.append(greedy_count / (step + 1))
    trajectory.append(epi_trajectory)
    coverage.append(env.get_coverage())

    if episode % 1000 == 0:
        q_class.append(copy.deepcopy(q))

    # update epsilon
    q.update_eps()

    print(episode, time_steps[episode], epsilons[episode], greedy[episode], coverage[episode])

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Streaming output truncated to the last 5000 lines.
195000 200 3.3949559024039117e-09 1.0 0.8585858585858586
195001 200 3.3946164068136713e-09 1.0 0.8585858585858586
195002 200 3.39427694517299e-09 1.0 0.8585858585858586
195003 200 3.3939375174784728e-09 1.0 0.8585858585858586
195004 200 3.3935981237267248e-09 1.0 0.8585858585858586
195005 200 3.393258763914352e-09 1.0 0.8585858585858586
195006 200 3.3929194380379606e-09 1.0 0.8585858585858586
195007 200 3.392580146094157e-09 1.0 0.8585858585858586
195008 200 3.3922408880795475e-09 1.0 0.8585858585858586
195009 200 3.3919016639907397e-09 1.0 0.8585858585858586
195010 200 3.391562473824341e-09 1.0 0.8585858585858586
195011 200 3.3912233175769586e-09 1.0 0.8585858585858586
195012 200 3.3908841952452007e-09 1.0 0.8585858585858586
195013 200 3.390545106825676e-09 1.0 0.8585858585858586
195014 200 3.3902060523149936e-09 1.0 0.8585858585858586
195015 200 3.3898670317097623e-09 1.0 0.8585858585858586
195016 200 3.3895280450065912e-09 1.0 0.858

In [7]:
idx = []
for i in range(10):
    for j in range(10):
        idx.append((i, j))

In [23]:
len(q_class)

200

In [31]:
df = pd.DataFrame(q_class[30-1].q_table, index=idx)
df.to_csv('qtable.csv')

In [14]:
import joblib

In [19]:
joblib.dump(q_class, "q_class_single_fixed.txt", compress=3)

['q_class.txt']

### Random Initial Position

In [32]:
# records for each episode
time_steps = [] # number of time steps in total
epsilons = [] # epsilon at the end of each episode
greedy = [] # the ratio of greedy choices
trajectory = []
coverage = []

q_class = []

# parameters for training
train_episodes = 200000
size = 10
max_steps = size * 20

# initialize the environment and the q tables
env = Grid(x_size=size)
q = QTables(observation_space=env.observation_space, action_space=env.action_space, eps_start=1, eps_end=0, gamma=0.5, r=0.9999, lr=0.01)

# training
for episode in range(train_episodes):
    env.reset()
    state = env.get_agent_obs()
    eps_tmp = q.eps

    greedy_count = 0
    epi_trajectory = []
    epi_step = 0
    epi_trajectory.append(env.get_agent_obs())

    for step in range(max_steps):
        action, greedy_tf = q.get_action(obs=state)
        next_state, reward, done = env.step(action)
        q.train(state, next_state, action, reward, done)

        greedy_count += greedy_tf * 1
        epi_trajectory.append(env.get_agent_obs())

        if done:
            epi_step = step
            break
    
        # update the observation
        state = next_state

    # record
    time_steps.append(len(epi_trajectory)-1)
    epsilons.append(eps_tmp)
    greedy.append(greedy_count / (step + 1))
    trajectory.append(epi_trajectory)
    coverage.append(env.get_coverage())

    if episode % 1000 == 0:
        q_class.append(copy.deepcopy(q))

    # update epsilon
    q.update_eps()

    print(episode, time_steps[episode], epsilons[episode], greedy[episode], coverage[episode])

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Streaming output truncated to the last 5000 lines.
195000 200 3.3949559024039117e-09 1.0 0.7171717171717171
195001 200 3.3946164068136713e-09 1.0 0.7171717171717171
195002 200 3.39427694517299e-09 1.0 0.7171717171717171
195003 200 3.3939375174784728e-09 1.0 0.7171717171717171
195004 200 3.3935981237267248e-09 1.0 0.7777777777777778
195005 200 3.393258763914352e-09 1.0 0.7171717171717171
195006 200 3.3929194380379606e-09 1.0 0.7373737373737373
195007 200 3.392580146094157e-09 1.0 0.7171717171717171
195008 200 3.3922408880795475e-09 1.0 0.7171717171717171
195009 200 3.3919016639907397e-09 1.0 0.7171717171717171
195010 200 3.391562473824341e-09 1.0 0.7171717171717171
195011 200 3.3912233175769586e-09 1.0 0.7171717171717171
195012 200 3.3908841952452007e-09 1.0 0.7373737373737373
195013 200 3.390545106825676e-09 1.0 0.7171717171717171
195014 200 3.3902060523149936e-09 1.0 0.7171717171717171
195015 200 3.3898670317097623e-09 1.0 0.7171717171717171
195016 200 3.3895280450065912e-09 1.0 0.717

In [33]:
joblib.dump(q_class, "q_class_single_random.txt", compress=3)

['q_class_single_random.txt']

In [41]:
df = pd.DataFrame(q_class[10-1].q_table, index=idx)
df.to_csv('qtable.csv')